# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 2:24PM,257477,16,9162687,Sartorius Bioprocess Solutions,Released
1,Feb 15 2023 2:03PM,257476,10,641595,"NBTY Global, Inc.",Released
2,Feb 15 2023 1:59PM,257475,10,MSP218583,"Methapharm, Inc.",Released
3,Feb 15 2023 1:59PM,257475,10,MSP218579,"Methapharm, Inc.",Released
4,Feb 15 2023 1:59PM,257475,10,MSP218580,"Methapharm, Inc.",Released
5,Feb 15 2023 1:59PM,257475,10,MSP218581,"Methapharm, Inc.",Released
6,Feb 15 2023 1:59PM,257475,10,MSP218582,"Methapharm, Inc.",Released
7,Feb 15 2023 1:49PM,257474,10,EYE43026,Eye Pharma Inc,Released
8,Feb 15 2023 1:42PM,257473,10,SOTAH0000910,"Nextsource Biotechnology, LLC",Released
9,Feb 15 2023 1:32PM,257471,10,0086400929,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,257473,Released,1
44,257474,Released,1
45,257475,Released,5
46,257476,Released,1
47,257477,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257473,NaN,1.0
257474,NaN,1.0
257475,NaN,5.0
257476,NaN,1.0
257477,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257232,16.0,8.0
257386,1.0,0.0
257387,1.0,1.0
257392,0.0,5.0
257396,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257232,16,8
257386,1,0
257387,1,1
257392,0,5
257396,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257232,16,8
1,257386,1,0
2,257387,1,1
3,257392,0,5
4,257396,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257232,16,8
1,257386,1,
2,257387,1,1
3,257392,,5
4,257396,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc."
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc."
7,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc
8,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC"
9,Feb 15 2023 1:32PM,257471,10,ISDIN Corporation
39,Feb 15 2023 1:31PM,257472,10,Emerginnova
42,Feb 15 2023 1:29PM,257470,10,ISDIN Corporation
43,Feb 15 2023 12:47PM,257468,10,"Citieffe, Inc."
44,Feb 15 2023 12:29PM,257466,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions,,1
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc.",,1
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc.",,5
3,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc,,1
4,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC",,1
5,Feb 15 2023 1:32PM,257471,10,ISDIN Corporation,,30
6,Feb 15 2023 1:31PM,257472,10,Emerginnova,,3
7,Feb 15 2023 1:29PM,257470,10,ISDIN Corporation,,1
8,Feb 15 2023 12:47PM,257468,10,"Citieffe, Inc.",,1
9,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions,1,
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc.",1,
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc.",5,
3,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc,1,
4,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC",1,
5,Feb 15 2023 1:32PM,257471,10,ISDIN Corporation,30,
6,Feb 15 2023 1:31PM,257472,10,Emerginnova,3,
7,Feb 15 2023 1:29PM,257470,10,ISDIN Corporation,1,
8,Feb 15 2023 12:47PM,257468,10,"Citieffe, Inc.",1,
9,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions,1,
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc.",1,
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc.",5,
3,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc,1,
4,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions,1.0,NaN
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc.",1.0,NaN
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc.",5.0,NaN
3,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc,1.0,NaN
4,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 15 2023 2:24PM,257477,16,Sartorius Bioprocess Solutions,1.0,0.0
1,Feb 15 2023 2:03PM,257476,10,"NBTY Global, Inc.",1.0,0.0
2,Feb 15 2023 1:59PM,257475,10,"Methapharm, Inc.",5.0,0.0
3,Feb 15 2023 1:49PM,257474,10,Eye Pharma Inc,1.0,0.0
4,Feb 15 2023 1:42PM,257473,10,"Nextsource Biotechnology, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6950934,220.0,1.0
12,514854,1.0,1.0
15,514695,16.0,32.0
16,514942,2.0,0.0
19,1287186,23.0,26.0
21,1029770,4.0,0.0
22,514789,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6950934,220.0,1.0
1,12,514854,1.0,1.0
2,15,514695,16.0,32.0
3,16,514942,2.0,0.0
4,19,1287186,23.0,26.0
5,21,1029770,4.0,0.0
6,22,514789,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,220.0,1.0
1,12,1.0,1.0
2,15,16.0,32.0
3,16,2.0,0.0
4,19,23.0,26.0
5,21,4.0,0.0
6,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,220.0
1,12,Released,1.0
2,15,Released,16.0
3,16,Released,2.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,1.0,1.0,32.0,0.0,26.0,0.0,1.0
Released,220.0,1.0,16.0,2.0,23.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,1.0,1.0,32.0,0.0,26.0,0.0,1.0
1,Released,220.0,1.0,16.0,2.0,23.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,1.0,1.0,32.0,0.0,26.0,0.0,1.0
1,Released,220.0,1.0,16.0,2.0,23.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()